In [ ]:
import spacy
import os
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
nlp = spacy.load("fr_core_news_sm")
brute_sentences_df = pd.read_csv('../data/dataset/brute_sentences.csv')
train_station_df = pd.read_csv('../data/liste-des-gares.csv', sep=';')

In [6]:
clean_df = []

# GOAL: make a IOB Tagging style dataset
# NEEDS: Loop on every train station and sentences (dataset of size 750k)
with tqdm(total=len(train_station_df) * len(brute_sentences_df), unit="train station") as pbar:
    for train_station in train_station_df["LIBELLE"]:
        departure = train_station
        sentences = []

        for sentence in brute_sentences_df["sentence"]:
            arrival = train_station_df.sample()
            new_sentence = sentence.replace('[departure]', departure)
            new_sentence = new_sentence.replace('[arrival]', arrival["LIBELLE"].values[0])
            doc_sentence = nlp(sentence)

            for word in [token_word.text for token_word in doc_sentence]:
                if (word in ["[departure]", "[arrival]"]):
                    if(word == "[departure]"):
                        actual_word = departure
                    else:
                        actual_word = arrival["LIBELLE"].values[0]

                    doc_word = nlp(word)
                    for sub_word_idx, sub_word in enumerate([token_sub_word.text for token_sub_word in doc_word]):
                        if (sub_word_idx == 0) :
                            clean_df.append(
                                {
                                    "word": actual_word,
                                    "status": "B"
                                }
                            )
                            continue
                        clean_df.append(
                            {
                                "word": actual_word,
                                "status": "I"
                            }
                        )
                        continue
                    clean_df.append(
                        {
                            "word": word,
                            "status": "O"
                        }
                    )

            pbar.update(1)

clean_df = pd.DataFrame(clean_df)

NameError: name 'tqdm' is not defined

In [39]:
clean_df_shuffled = clean_df.sample(frac=1)
train_df = clean_df_shuffled[:int(len(clean_df_shuffled)*0.7)]
valid_df = clean_df_shuffled[int(len(clean_df_shuffled)*0.7):int(len(clean_df_shuffled)*0.85)]
test_df = clean_df_shuffled[int(len(clean_df_shuffled)*0.85):]

In [40]:
train_df.to_csv('./data/dataset/departure_train.csv', index=False)
valid_df.to_csv('./data/dataset/departure_valid.csv', index=False)
test_df.to_csv('./data/dataset/departure_test.csv', index=False)